# Reviewing the Covid19 Tweet Dataset
https://github.com/thepanacealab/covid19_twitter

In [2]:
import pandas as pd

%load_ext lab_black
%matplotlib inline

In [28]:
from glob import glob
from tqdm import tqdm

# Data To Gather
1. Counts of Tweets Per Minute
    - Clean dataset
    - Raw Dataset
2. Top 1000 Terms / Day
3. "" Bigrams
4. "" Trigrams

# Covid19 Tweet Counts
- Clean removes reweets
- Raw includes everything

In [79]:
dss = glob("covid19_twitter/dailies/*/*-dataset.tsv.gz")

In [71]:
dss_clean = [c for c in dss if "clean" in c]
dss_raw = [c for c in dss if "clean" not in c]

In [142]:
def by_minute_stats(filename):
    """
    Read in a file from the covid-19 twitter chatter dataset
    and format to by minute stats
    """
    df = pd.read_csv(filename, sep="\t")
    df["datetime"] = df["date"] + " " + df["time"]
    df["datetime"] = pd.to_datetime(df["datetime"])
    df["datetime_round"] = df["datetime"].round("60s")
    if "country_code" in df.columns:
        df["country_code"] = df["country_code"].fillna("None")
        df_counts = (
            df.groupby(["datetime_round", "lang", "country_code"])
            .size()
            .reset_index()
            .rename(columns={0: "raw_count"})
        )
    else:
        df_counts = (
            df.groupby(["datetime_round"])
            .size()
            .reset_index()
            .rename(columns={0: "raw_count"})
        )
    return df_counts

In [143]:
# Read all Raw Tweet Counts
all_counts = []
for ds in tqdm(dss_raw):
    df_counts = by_minute_stats(ds)
    all_counts.append(df_counts)
all_counts = pd.concat(all_counts)

 14%|█▍        | 86/624 [02:38<18:54,  2.11s/it]/home/robmulla/anaconda3/envs/kaggle/lib/python3.7/site-packages/ipykernel_launcher.py:4: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  after removing the cwd from sys.path.
100%|██████████| 624/624 [18:13<00:00,  1.75s/it]


In [147]:
all_counts.to_csv("covid_tweet_chatter_raw.csv", index=False)

In [148]:
all_counts.to_parquet("covid_tweet_chatter_raw.parquet")